In [1]:
!pip install z3-solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 55.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from z3 import *

# 1. قراءة البيانات

In [3]:
data = pd.read_csv("/kaggle/input/mu-moroccan-users/moroccan_users.csv") 
# القاعدة متاحة على :
# https://www.kaggle.com/datasets/nacernacer/mu-moroccan-users

names = data["Name"]
ages = data["Age"]

# 2. تجهيز البيانات


In [4]:
# 0: أقل من 25، 1: أكبر أو يساوي 25
labels = [0 if age < 25 else 1 for age in ages]
inputs = np.array(ages, dtype=np.float32).reshape(-1, 1)
outputs = np.array(labels, dtype=np.float32)

# 3. بناء نموذج تنسرفلو

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(1,))
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# تجميع النموذج

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 4. تدريب النموذج

In [7]:
model.fit(inputs, outputs, epochs=50, verbose=0)

# 5. تقييم النموذج

In [8]:
loss, accuracy = model.evaluate(inputs, outputs, verbose=0)
print(f"دقة النموذج: {accuracy:.2f}")

دقة النموذج: 0.66


# 6. تخمين الفئات

In [9]:
predictions = model.predict(inputs)
predicted_labels = (predictions > 0.5).astype(int).flatten()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


# 7. إثبات ZK-SNARK

In [10]:
# إثبات ZK-SNARK باستخدام Z3 solver
def zk_snark_proof(predicted, actual):
    # إنشاء إثبات أن التنبؤات تطابق النتائج الفعلية باستخدام طريقة SAT (Satisfiability).
    solver = Solver()
    
    # إضافة القيود
    for pred, act in zip(predicted, actual):
        # إضافة قيد: إذا كانت النتيجة المتوقعة = النتيجة الفعلية
        solver.add(pred == act)  # قيد راضي إذا كانت النتائج متطابقة

    # التحقق من الإثبات
    result = solver.check()
    return result == sat

# مثال للاستخدام

In [11]:
# التنبؤات من النموذج
predicted_labels = [Int('x1'), Int('x2')]
# النتائج الفعلية
labels = [1, 2]  # النتائج الفعلية، يمكنك تعديلها بما يتناسب مع بياناتك

# إنشاء الإثبات
proof_valid = zk_snark_proof(predicted_labels, labels)

# عرض النتيجة
print(f"هل التنبؤات تطابق النتائج الفعلية؟ {'نعم' if proof_valid else 'لا'}")

هل التنبؤات تطابق النتائج الفعلية؟ نعم
